# 1. 클래스 알아보기

## 상속관계 

In [ ]:
type P struct {

}

In [ ]:
type A struct {
    P
}

In [2]:
class P :
    def ppp(self) :
        print(" parents ")

In [11]:
class A(P):
    def ppp(self) :
        P.ppp(self)
        print(" child ")

### 상속관계 확인 

In [8]:
A.mro()

[__main__.A, __main__.P, object]

In [13]:
super(A,A()).ppp

<bound method P.ppp of <__main__.A object at 0x7ff34f979f10>>

In [12]:
super(P,A()).__new__

<function object.__new__(*args, **kwargs)>

In [10]:
A().ppp()

 parents 
 child 


###  부모 클래스 내의 멤버 사용하기

- super를 객체로 만들어서 사용 

In [6]:
class S :
    def sss(self) :
        print(" S parents ")

In [7]:
class A_(P,S) :
    def ppp(self) :
        super().ppp()
        super(P,A_()).sss()
        print(" child ")
        

In [8]:
A_.mro()

[__main__.A_, __main__.P, __main__.S, object]

In [9]:
A_().ppp()

 parents 
 S parents 
 child 


## 상속관계 : 믹스인 처리

# 2. 메타 클래스 알아보기

##  내장 클래스 상속관계

In [10]:
isinstance(type, object)

True

In [11]:
issubclass(type, object)

True

In [12]:
isinstance( object, type)

True

In [13]:
issubclass(object,type)

False

In [14]:
help(type.__call__)

Help on wrapper_descriptor:

__call__(self, /, *args, **kwargs)
    Call self as a function.



In [15]:
help(object.__call__)

Help on method-wrapper object:

__call__ = class method-wrapper(object)
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __reduce__(...)
 |      Helper for pickle.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __objclass__
 |  
 |  __self__
 |  
 |  __text_signature__



In [16]:
object.__call__

<method-wrapper '__call__' of type object at 0x10d1ae218>

##  사용자 메타 클래스 만들기

In [17]:
class UserType_2(type) :
    
    def __new__(meta,name,bases,namespace) :
        print('-----------------------------------')
        print ( "Allocating memory for class", name)
        print( meta)
        print( bases)
        print( namespace)
        
        instance = super().__new__(meta,name,bases,namespace)
        return instance
    
    def __init__(cls,name,bases,namespace) :
        print('-----------------------------------')
        print ( "Allocating memory for class", name)
        print( cls)
        print( bases)
        print( namespace)
        
        super(UserType_2, cls).__init__(name, bases, namespace)
        
    def __call__(self,*args) :
        print(" instance call")
        ins = self.__new__(self,*args)
        self.__init__(ins, *args)

### 새로운 클래스 만들기

- 매개변수는 클래스이름(문자열), 상속관계(튜플), 네임스페이스(딕셔너리)

In [18]:
AB = UserType_2('AB',(object,),{})

-----------------------------------
Allocating memory for class AB
<class '__main__.UserType_2'>
(<class 'object'>,)
{}
-----------------------------------
Allocating memory for class AB
<class '__main__.AB'>
(<class 'object'>,)
{}


In [19]:
AB

__main__.AB

In [20]:
ab = AB()

 instance call


In [21]:
ab

### 메타 클래스로 직접 사용해서 처리하기

In [22]:
AB_ = type.__call__(UserType_2,'AB_',(object,),{})

-----------------------------------
Allocating memory for class AB_
<class '__main__.UserType_2'>
(<class 'object'>,)
{}
-----------------------------------
Allocating memory for class AB_
<class '__main__.AB_'>
(<class 'object'>,)
{}


In [23]:
AB_

__main__.AB_

## 메타 클래스를 사용해서 클래스 만들기

In [24]:
class UserType(type) :
      def __call__(self,*args) :
        ins = self.__new__(self,*args)
        self.__init__(ins, *args)
        return ins

### 클래스 상속관계 내부에 metaclass 매개변수에 메타클래스 정의

In [25]:
class A(metaclass=UserType) :
    def __new__(cls,x,y) :
        return super().__new__(cls)
    
    def __init__(self, x, y) :
        self.x = x
        self.y = y
        
    def __call__(self) :
        print(" user defined class instance call")

In [26]:
a = A(1,2)

In [27]:
a.__dict__

{'x': 1, 'y': 2}

## 싱글턴 객체 만들기 

In [28]:
class Single :
    _instance = None
    def __new__(cls,*args) :
        print(" class ", cls)
        if Single._instance == None :
            Single._instance = super().__new__(cls)
        
        return Single._instance
    
    def __init__(self, x, y) :
        self.x = x
        self.y = y

In [29]:
s = Single(10,20)

 class  <class '__main__.Single'>


In [30]:
s

In [31]:
s1 = Single(10,20)

 class  <class '__main__.Single'>


In [32]:
s1

In [33]:
s == s1

True

### 내장함수를 사용해서 속성에 객체를 저장하도록 처리

In [14]:
class PP :
    a = 100

In [16]:
PP.__dict__['a']

100

In [15]:
getattr(PP,"a")

100

In [18]:
setattr(PP,'a',99)

In [19]:
PP.__dict__['a']

99

In [20]:
delattr(PP,'a')

In [24]:
dir(dict)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [27]:
PP.a =  99999

In [28]:
PP.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'PP' objects>,
              '__weakref__': <attribute '__weakref__' of 'PP' objects>,
              '__doc__': None,
              'a': 99999})

In [1]:
class Singleton :
    def __new__(cls,*args) :
        print(" class ", cls)
        if not hasattr(Singleton,"_instance") :
            Singleton._instance = super().__new__(cls)
            Singleton._count = 1
        
        return getattr(Singleton, "_instance")
    
    def __init__(self, name) :
        if not hasattr(self,"name")  :
            self.name = name
            
    def __setattr__(self,key,value) :     ## 기존 값만 갱신하기 
        if not hasattr(self,key)  :
            super().__setattr__(key,value)
        

In [2]:
s = Singleton("sol")

 class  <class '__main__.Singleton'>


In [3]:
s.name

'sol'

In [4]:
s.name = "Moon"

In [5]:
s.name

'sol'

In [39]:
s1 = Singleton("jong")

 class  <class '__main__.Singleton'>


In [40]:
s1.name

'sol'

In [30]:
Singleton.mro()

[__main__.Singleton, object]

In [35]:
ss = Singleton()

 class  <class '__main__.Singleton'>


In [36]:
ss

In [37]:
ss1 = Singleton()

 class  <class '__main__.Singleton'>


In [38]:
ss1

In [39]:
ss == ss1

True